In [6]:
import numpy as np
import pandas as pd
from mido import MidiFile

In [7]:
midi_dir = "../midi/"

In [9]:
# Load a midi file
mid = MidiFile(midi_dir + "Fr_Elise.mid")

In [10]:
# Print all messages for each track in midi
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: Piano 
MetaMessage('track_name', name='Piano\x00', time=0)
MetaMessage('time_signature', numerator=1, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('set_tempo', tempo=833333, time=0)
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
MetaMessage('midi_port', port=0, time=0)
note_on channel=0 note=76 velocity=33 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=75 velocity=33 time=7
note_on channel=0 note=75 velocity=0 time=113
MetaMessage('time_signature', numerator=3, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=7)
note_on channel=0 note=76 velocity=33 time=0
note_on channel=0 note=76 velocity=